# Diccionario de datos

In [76]:
import pandas as pd
from pathlib import Path
import numpy as np
import json

In [77]:
EXCEL_PATH = "../data/Base infecciones POPTH.xlsx"
CLEAN_EXCEL_PATH = "../data/Base_infecciones_POPTH_limpia.xlsx"
CODEBOOK_PATH = "../data/Codebook.json"


Cargamos los datos ya limpios para crear el diccionario de datos.

In [78]:
df = pd.read_excel(CLEAN_EXCEL_PATH, engine="openpyxl")
df

,Código anonimizado,#Paciente_Tx,Año_Tx,Fecha_Tx,Etiología_#1,Etiología_#2,Edad,Sexo,Child_Pugh_Score_Cat,Child_Pugh_Score,...,Requerimiento_de_diálisis,Días_En_UCI_POP,Trasfusión_GRE_hasta_1m_POP,Reintervención_Quirúrgica_hasta_1m_POP,Retrasplante,Vivo_Hoy,Fecha_Control/Muerte,SOBREVIDA_DIAS,SOBREVIDA_MESES,SOBREVIDA_AÑOS
0,P001,1,2009,2009-01-03,1,0,62,1,NaN,NaN,...,0,3.0,0,1.0,0.0,0,2009-10-05,275,9.17,0.76
1,P002,2,2009,2009-01-10,3,0,62,1,NaN,NaN,...,0,1.0,0,1.0,0.0,0,2009-01-11,1,0.03,0.00
2,P003,3,2009,2009-02-03,8,0,56,2,NaN,NaN,...,0,7.0,0,0.0,0.0,1,2023-12-31,5444,181.47,15.12
3,P004,4,2009,2009-02-05,3,0,58,1,NaN,NaN,...,0,5.0,0,0.0,0.0,1,2023-12-31,5442,181.40,15.12
4,P005,5,2009,2009-02-11,4,0,62,2,NaN,NaN,...,0,3.0,0,0.0,0.0,1,2023-12-31,4724,157.47,13.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,P553,553,2024,2024-09-21,12,0,22,1,C,11.0,...,0,0.0,0,0.0,0.0,1,2024-09-21,0,0.00,0.00
553,P554,554,2024,2024-09-21,8,0,33,2,C,10.0,...,1,NaN,1,NaN,NaN,0,2025-03-31,174,5.00,0.00
554,P555,555,2024,2024-10-08,3,4,55,1,B,9.0,...,0,0.0,0,0.0,0.0,1,2025-05-07,203,6.00,0.00
555,P556,556,2024,2024-10-16,27,0,54,2,B,9.0,...,0,0.0,0,0.0,0.0,1,2025-05-13,196,6.00,0.00


In [79]:
def cast_numeric_best(df: pd.DataFrame):
    """
    For each column:
      - If all non-null values are numeric, convert to:
          - Int64 (nullable) if all numeric values are integers.
          - Float64 (nullable) if any numeric value has a fractional part.
      - If any non-numeric value is present, keep the column as object.
 
    Returns
    -------
    (DataFrame, list)
        DataFrame with converted dtypes, and a list of columns that could not
        be safely converted to numeric.
    """
    df_out = df.copy()
    non_converted_columns = []
 
    for col in df_out.columns:
        s = df_out[col]

        # fecha dejar como está
        if pd.api.types.is_datetime64_any_dtype(s):
            continue
 
        # If already numeric, try to refine to Int64 where possible
        if pd.api.types.is_numeric_dtype(s):
            # converts non-numeric values to NaN
            sn = pd.to_numeric(s, errors="coerce")
            # check for fractional parts, if any, use Float64
            frac = sn.dropna() != np.floor(sn.dropna())
            if not frac.any():
                df_out[col] = sn.astype("Int64")   # integers with NA
            else:
                df_out[col] = sn.astype("Float64")  # floats with NA
 
        else:
            # For non-numeric/object, try converting non-null values to numeric
            sn_non_null = pd.to_numeric(s[s.notna()], errors="coerce")
 
            if sn_non_null.notna().all():
                # All non-null values are numeric
                frac = sn_non_null.dropna() != np.floor(sn_non_null.dropna())
                sn_full = pd.to_numeric(s, errors="coerce")
                if not frac.any():
                    df_out[col] = sn_full.astype("Int64")
                else:
                    df_out[col] = sn_full.astype("Float64")
            else:
                # There are values that cannot be converted -> keep as object
                non_converted_columns.append(col)
                # normalize objects
                df_out[col] = s.astype("string")
 
    return df_out, non_converted_columns
 

In [80]:
df, non_converted = cast_numeric_best(df)

In [81]:
df.dtypes.value_counts()

Int64             49
Float64            6
datetime64[ns]     4
string[python]     2
Name: count, dtype: int64

En el archivo excel entregado por el cliente, se encuentra una hoja llamada "Codificación" que contiene la descripción de cada variable en el dataset. En caso de ser una variable categorica, incluye la codificación. A continuación se carga dicha hoja de excel.

In [82]:
df_codebook_original = pd.read_excel(EXCEL_PATH, sheet_name="Codificación")

In [83]:
df_codebook_original.columns = df_codebook_original.columns.str.strip()

In [84]:
print("Número de columnas(variables):", df_codebook_original.shape[1])

Número de columnas(variables): 69


Ahora revisamos la estructura del dataframe con la codificación

In [85]:
df_codebook_original

,#Paciente_Tx,Año_Tx,Fecha_Tx,ID,Nombre,Edad,Sexo,Etiología_#1,Etiología_#2,Child_Pugh_Score,...,SOBREVIDA_DIAS,SOBREVIDA_MESES,SOBREVIDA_AÑOS,#,ID.1,Nombre,Infección POP,Tipo de infección,Aislamiento,Fecha
0,Interpretación,Año del trasplante,Tiempo transcurrido \ndesde el trasplante \nhe...,ID del paciente,Nombre del paciente,Tiempo transcurrido \na partir del \nnacimient...,"Condición orgánica, \nmasculina o \nfemenina",Enfermedad que \nlo llevó a la \nenfermedad he...,Enfermedad que \nlo llevó a la \nenfermedad he...,Child_Pugh_Score,...,Sobrevida en días post trasplante,Sobrevida en meses post trasplante,Sobrevida en años post trasplante,0.0,NaN,NaN,No,NaN,NaN,NaN
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NINGUNA,NINGUNA,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,Sí,ISO,K. PNEUMONIAE,NaN
2,1,NaN,NaN,NaN,NaN,NaN,M,HCV,HCV,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,IVU,CMV,NaN
3,2,NaN,NaN,NaN,NaN,NaN,F,HBV,HBV,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NEUMONÍA,ENTEROCOCO FAECIUM,NaN
4,3,NaN,NaN,NaN,NaN,NaN,NaN,NASH,NASH,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,CVC,STREPTOCOCCUS VIRIDANS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 Revisando se encontraron los siguientes problemas:
 - Hay más variables en el codebook entregado que en el dataset.
 - Hay algunas variables en el codebook que no existen en el dataset entregado o que tienen nombres diferentes.
 - No todas las variables categoricas incluyen la descripción de la codificaicón. Y en muchos casos no la siguen.



Teniendo en cuenta lo anterior, se procedió a crear un nuevo diccionario de datos.

In [86]:
# obtener la primera fila del DataFrame, contiene las descripciones
df_original = pd.read_excel(EXCEL_PATH, sheet_name="Base de datos principal")
df_original.columns = df_original.columns.str.strip()
first_row = df_original.iloc[0].reset_index()

In [87]:
df_codebook_new = first_row.copy()
df_codebook_new.columns = ["Variable", "Description"]
df_codebook_new["Variable"] = df_codebook_new["Variable"].str.strip()
df_codebook_new["Description"] = df_codebook_new["Description"].str.strip().str.replace("\n", "").str.lower().str.replace("uci", "UCI").str.replace("tx", "transplante")
df_codebook_new["Dtype"] = df_codebook_new["Variable"].map(df.dtypes).astype(str)
df_codebook_new

,Variable,Description,Dtype
0,#Paciente_Tx,número de paciente,Int64
1,Año_Tx,año del trasplante,Int64
2,Fecha_Tx,tiempo transcurrido desde el trasplante hepático,datetime64[ns]
3,Código anonimizado,código anonimizado para tratamiento de datos d...,string
4,Edad,tiempo transcurrido a partir del nacimiento de...,Int64
5,Sexo,"condición orgánica, masculina o femenina",Int64
6,Etiología_#1,enfermedad que lo llevó a la enfermedad hepática,Int64
7,Etiología_#2,enfermedad que lo llevó a la enfermedad hepática,Int64
8,Child_Pugh_Score,child_pugh_score,Int64
9,Meld_Score,meld_score,Int64


In [88]:
def creat_dic(col_name, df_codebook_original):
    dic_column = {}
    # Solo intentar si la columna existe en el codebook original
    if col_name in df_codebook_original.columns:
        df_column = df_codebook_original[col_name].loc[1:]
        df_column.index = df_column.index-1
        dic_column = df_column.to_dict()
        # Elimina del diccionario todas las llaves cuyos valores son NaN.
        dic_column = {key: value for key, value in dic_column.items() if not pd.isna(value)}

    else:
        print(f"Columna '{col_name}' no encontrada en el codebook original.")
    return dic_column


# 2. Aplicar la lógica de mapeo y limpieza en un solo paso
df_codebook_new["Choices"] = df_codebook_new["Variable"].map(lambda col: creat_dic(col, df_codebook_original))
df_codebook_new.at[0, "Choices"] = {}

# 3. Mostrar el resultado
display(df_codebook_new)

Columna 'Código anonimizado' no encontrada en el codebook original.
Columna 'Tiempo_Isquemia_caliente' no encontrada en el codebook original.


,Variable,Description,Dtype,Choices
0,#Paciente_Tx,número de paciente,Int64,{}
1,Año_Tx,año del trasplante,Int64,{}
2,Fecha_Tx,tiempo transcurrido desde el trasplante hepático,datetime64[ns],{}
3,Código anonimizado,código anonimizado para tratamiento de datos d...,string,{}
4,Edad,tiempo transcurrido a partir del nacimiento de...,Int64,{}
5,Sexo,"condición orgánica, masculina o femenina",Int64,"{1: 'M', 2: 'F'}"
6,Etiología_#1,enfermedad que lo llevó a la enfermedad hepática,Int64,"{0: 'NINGUNA', 1: 'HCV', 2: 'HBV', 3: 'NASH', ..."
7,Etiología_#2,enfermedad que lo llevó a la enfermedad hepática,Int64,"{0: 'NINGUNA', 1: 'HCV', 2: 'HBV', 3: 'NASH', ..."
8,Child_Pugh_Score,child_pugh_score,Int64,{}
9,Meld_Score,meld_score,Int64,{}


# Tipo de variable

In [89]:
df_codebook_new["Var Type"] = "" # placeholder
df_codebook_new.loc[df_codebook_new["Variable"] == "Código anonimizado", "Var Type"] = "Texto"

In [90]:
datetime_vars = df_codebook_new["Dtype"].apply(lambda x: x.lower().startswith("datetime"))
int_vars = df_codebook_new["Dtype"].apply(lambda x: x.lower().startswith("int"))
float_vars = df_codebook_new["Dtype"].apply(lambda x: x.lower().startswith("float"))
numerical_vars = (df_codebook_new["Choices"] == {}) & (~datetime_vars)
discrete_vars = int_vars & numerical_vars
continuos_vars = float_vars & numerical_vars
categorical_vars = df_codebook_new["Choices"] != {}

In [91]:
df_codebook_new.loc[datetime_vars, "Var Type"] = "Fecha"
df_codebook_new.loc[discrete_vars, "Var Type"] = "Númerica Discreta"
df_codebook_new.loc[continuos_vars, "Var Type"] = "Númerica Continua"
df_codebook_new.loc[categorical_vars, "Var Type"] = "Categórica"

In [92]:
df_codebook_new = df_codebook_new.rename(columns={
    "Description": "Descripción",
    "Var Type": "Tipo de Variable",
    "Choices": "Valores"
})

In [93]:
df_codebook_new = df_codebook_new[["Variable", "Descripción", "Tipo de Variable", "Valores"]]#, "Dtype"]]

In [94]:
df_codebook_new

,Variable,Descripción,Tipo de Variable,Valores
0,#Paciente_Tx,número de paciente,Númerica Discreta,{}
1,Año_Tx,año del trasplante,Númerica Discreta,{}
2,Fecha_Tx,tiempo transcurrido desde el trasplante hepático,Fecha,{}
3,Código anonimizado,código anonimizado para tratamiento de datos d...,Texto,{}
4,Edad,tiempo transcurrido a partir del nacimiento de...,Númerica Discreta,{}
5,Sexo,"condición orgánica, masculina o femenina",Categórica,"{1: 'M', 2: 'F'}"
6,Etiología_#1,enfermedad que lo llevó a la enfermedad hepática,Categórica,"{0: 'NINGUNA', 1: 'HCV', 2: 'HBV', 3: 'NASH', ..."
7,Etiología_#2,enfermedad que lo llevó a la enfermedad hepática,Categórica,"{0: 'NINGUNA', 1: 'HCV', 2: 'HBV', 3: 'NASH', ..."
8,Child_Pugh_Score,child_pugh_score,Númerica Discreta,{}
9,Meld_Score,meld_score,Númerica Discreta,{}


In [95]:
df_codebook_new["Tipo de Variable"].value_counts()

Tipo de Variable
Categórica           31
Númerica Discreta    18
Númerica Continua     6
Fecha                 4
Texto                 1
Name: count, dtype: int64

In [96]:
df_codebook_new[continuos_vars]

,Variable,Descripción,Tipo de Variable,Valores
14,Peso_previo_Cx,peso en kilogramos,Númerica Continua,{}
16,IMC,indice de masa corporal,Númerica Continua,{}
23,Tiempo_Cx,tiempo en horas,Númerica Continua,{}
24,Tiempo_Isquemia_Fria,tiempo de isquemia fría reportado en la histor...,Númerica Continua,{}
58,SOBREVIDA_MESES,sobrevida en meses post trasplante,Númerica Continua,{}
59,SOBREVIDA_AÑOS,sobrevida en años post trasplante,Númerica Continua,{}


In [97]:
orden = ["Código anonimizado", "#Paciente_Tx", "Año_Tx", "Fecha_Tx",	"Etiología_#1",	"Etiología_#2",	"Edad",	"Sexo",	
         "Child_Pugh_Score_Cat", "Child_Pugh_Score",	"Meld_Score","Diabetes_Mellitus","Tabaquismo","Alcoholismo",
        "Hipertensión_Arterial", "Peso_previo_Cx",	"Talla",	"IMC",	"Antecedente_UCI_6_meses_PreOP", "UCI_al_Momento_Tx",	
        "Tiempo_Isquemia_Fria",	"Tiempo_Isquemia_caliente",	"Tiempo_Cx", "Tipo_Reconstrucción_Biliar",	
        "Infección_Al_Momento_Tx",	"Localización de la infección",	"Germen/Microorganismo aislado",	
        "Antibiotico_previo_al_Tx",	"Días_Tratamiento_Antibiótico_Previo_A_Tx#1",	
        "Antibiótico_Profiláctico_VS_Terapéutico",	"Antibiótico_1__Tx#1",	"Antibiótico_2_Tx#1",	
        "Antibiótico_3_Tx#1", "Antifúngico_Tx#1",	"Tiempo_De_Dosis_Hasta_Tx#1",	
        "Nutrición_Enteral","Días_Nutrición_Enteral",	"Complicaciones_Técnicas",	"Dias_Estancia_Hospitalaria",	
        "Días_Hospitalización_UCI",	"Días_En_Hospitalización_Piso",	"Días_Totales_Intrahospitalarios",	
        "Inmunosupresión_con_Anticuerpos",	"Inmunosupresor_1_Postx",	"Inmunosupresor_2_PostTx",	
        "Inmunosupresor_1_1mesPostTx",	"Inmunosupresor_2_PostTx_1mesPx",	"Inmunosupresor_1_6mesesPostx",	
        "Inmunosupresor_2_6mesesPostTx", "Fecha_Egreso_UCI",	"Fecha_Egreso_Hospitalario",	
        "Requerimiento_de_diálisis", "Días_En_UCI_POP",	"Trasfusión_GRE_hasta_1m_POP",	
        "Reintervención_Quirúrgica_hasta_1m_POP",	"Retrasplante",	
        "Vivo_Hoy",	"Fecha_Control/Muerte",	"SOBREVIDA_DIAS",	"SOBREVIDA_MESES",	"SOBREVIDA_AÑOS"]
df_codebook = df_codebook_new.set_index("Variable")
df_codebook = df_codebook.loc[df_codebook.index.isin(orden)]
df_codebook = df_codebook.reindex(orden).reset_index()

In [98]:
df_codebook

,Variable,Descripción,Tipo de Variable,Valores
0,Código anonimizado,código anonimizado para tratamiento de datos d...,Texto,{}
1,#Paciente_Tx,número de paciente,Númerica Discreta,{}
2,Año_Tx,año del trasplante,Númerica Discreta,{}
3,Fecha_Tx,tiempo transcurrido desde el trasplante hepático,Fecha,{}
4,Etiología_#1,enfermedad que lo llevó a la enfermedad hepática,Categórica,"{0: 'NINGUNA', 1: 'HCV', 2: 'HBV', 3: 'NASH', ..."
...,...,...,...,...
56,Vivo_Hoy,define si está vivo hoy,Categórica,"{0: 'NO', 1: 'SI'}"
57,Fecha_Control/Muerte,fecha de control/muerte,Fecha,{}
58,SOBREVIDA_DIAS,sobrevida en días post trasplante,Númerica Discreta,{}
59,SOBREVIDA_MESES,sobrevida en meses post trasplante,Númerica Continua,{}


In [99]:
df_codebook.to_json(CODEBOOK_PATH, orient="records", force_ascii=False, indent=4)

In [100]:
df_codebook.to_excel("../data/codificacion.xlsx", index=False)